<a href="https://colab.research.google.com/github/edenmoz22/HW2P-Final_Project/blob/main/final_lettersModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/SitaraAlayev/HW2P-Final_Project.git

fatal: destination path 'HW2P-Final_Project' already exists and is not an empty directory.


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy



In [ ]:
PATH = '/content/HW2P-Final_Project/'

In [ ]:
dir_train = PATH + "dataset/train/"
dir_test = PATH + "dataset/test/"

log_dir = PATH + "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")


In [ ]:
def load_images(path, direc, img_size=28):
    # load all images from a specific folder and 
    # returns a X,y set of np.arrays
    X = []
    for f_name in os.listdir(path+direc):
        # define file full path
        file_path = path+direc+'/'+f_name
        # read and image
        img = cv2.imread(file_path, 0)
        # append image to X
        img = cv2.resize(img,(img_size,img_size))
        X.append(img)
        

    # turn X to np.array
    X = np.array(X)
    # get a label vector
    y = np.array([direc]*X.shape[0])
    
    return X, y

def load_directories(root_dir):
    # run over all folders, load X,y (images and labels), and save to a X,y pair for all data
    X = None
    for direc in os.listdir(root_dir):
        X_dir, y_dir = load_images(root_dir, direc)
        if X is None:
            X = X_dir.copy()
            y = y_dir.copy()
            continue
        X = np.append(X, X_dir, axis=0)
        y = np.append(y, y_dir)
        # pdb.set_trace()
    return X,y

# label to letter translation dictionary
label_to_letter = {str(i): letter for i, letter in list(enumerate('אבגדהוזחטיכךלמםנןסעפףצץקרשת'))}

# load X and y data (Images and Labels)
X_train, y_train = load_directories(dir_train)
X_test, y_test = load_directories(dir_test)



In [ ]:
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train)

In [ ]:
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
# Create a neural network model
def get_model(input_shape = (28,28,1), loss_function='categorical_crossentropy', 
                optimizer='rmsprop', no_classes=27):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(no_classes, activation='softmax'))

    # Compile the model
    model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
from sklearn.model_selection import KFold
# Merge inputs and targets
inputs = X_train
targets = y_train
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

# K-fold Cross Validation model evaluation
acc_per_fold, loss_per_fold= [], []
fold_no = 1
for train, test in kfold.split(inputs, targets):
    # Define the model architecture
    model = get_model()
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    n_eval = 20
    # Fit data to model
    for i in range(n_eval):
        history = model.fit(inputs[train], targets[train], batch_size=512, epochs=100//n_eval, verbose=1)
        scores = model.evaluate(inputs[test], targets[test], verbose=0)
    # Generate generalization metrics
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]:.3f}; {model.metrics_names[1]} of {scores[1]*100:.2f}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    # Increase fold number
    fold_no = fold_no + 1

In [ ]:
t_loss, a_acc = model.evaluate(X_test, y_test)

53/53 [==============================] - 0s 4ms/step - loss: 0.5682 - accuracy: 0.8659


In [ ]:
# Saving the model
with tf.device('/cpu:0'):
    model.save('lettersModel.h5')